In [19]:
import sys
import math
import pandas as pd
import altair as alt
import networkx as nx
import numpy as np

sys.path.append(r"..")

from mcroute import Network, StateSpace
import mcroute.matrix as matrix
import mcroute.vector as vector

In [20]:
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
edges = pd.read_csv('data/test_edges.csv')
ss = StateSpace.from_range(-10, 60)
I = matrix.identity(ss)
N = Network(ss)
for node in nodes:
    N.add_node(node, I)

for idx, edge in edges.iterrows():
    mx = matrix.truncnorm(ss, mean=edge['mean'], std=edge['std'])
    N.add_edge(edge['start'], edge['end'], mx, data={
        'tt': edge['mean'],
        'ttsd': edge['mean'] + 1.5*edge['std'],
        'tt2sd': edge['mean'] + 2*edge['std']
    })


In [21]:
path = nx.shortest_path(N, source='A', target='B', weight='tt')
path

['A', 'C', 'D', 'E', 'B']

In [22]:
path = nx.shortest_path(N, source='A', target='B', weight='ttsd')
path

['A', 'C', 'D', 'E', 'B']

In [23]:
path = nx.shortest_path(N, source='A', target='B', weight='tt2sd')
path

['A', 'B']

In [26]:
paths = nx.all_simple_paths(N, source='A', target='B')
p0 = vector.unit(ss, '0')
stats = []
thres = 0.95
for path in paths:
    sm = 0
    vecs = N.traverse(path, p0)
    average = np.average(ss.values, weights=vecs[-1])
    # Fast and numerically precise:
    std = math.sqrt(np.average((ss.values-average)**2, weights=vecs[-1]))
    for i in range(len(vecs[-1])):
        if sm >= thres:
            top = ss.values[i]
            break
        sm += vecs[-1][i]
    stats.append([''.join(path), round(average,2), round(std, 2), top])
df = pd.DataFrame(stats, columns=['Path', 'Mean', 'Std', 'p95'])
df.sort_values(by='Mean').style.hide_index()

Path,Mean,Std,p95
ACDEB,27.000000,3.980000,35.000000
AFGB,28.500000,3.500000,35.000000
AFGDEB,30.500000,5.350000,40.000000
AB,30.500000,4.010000,38.000000
ACFGB,31.000000,3.550000,38.000000
ACFGDEB,33.000000,5.390000,43.000000


In [28]:
df = pd.DataFrame(vecs[-1], columns=['prob']).reset_index()
alt.Chart(df).mark_line().encode(
    alt.X('index:O'),
    alt.Y('prob:Q')
)

alt.Chart(...)

In [8]:
for i in range(len(vecs[-1])):
    if sm >= thres:
        print(ss.values[i])
        break
    sm += vecs[-1][i]

-10.0


[[-0.75  0.45  0.3 ]
 [ 0.13 -0.67  0.64]
 [ 0.2   0.6  -0.8 ]]
[1. 1. 1.]
[[-0.75  0.13  0.2 ]
 [ 0.45 -0.67  0.6 ]
 [ 0.3   0.64 -0.8 ]
 [ 1.    1.    1.  ]]
[0. 0. 0. 1.]


array([0.16298395, 0.44067177, 0.39569553])